In [ ]:
!python /content/drive/MyDrive/causality-guided-Transformer/TES_Main_MAVEN_ERE.py \
-data /content/drive/MyDrive/causality-guided-Transformer/data/MAVEN_ERE/ \
-epoch 5 \
-batch_size 32 \
-d_model 512 \
-d_inner 256 \
-d_k 256 \
-d_v 256 \
-n_head 4 \
-n_layers 4 \
-dropout 0.1 \
-lr 1e-4\
-event_interest 7

In [ ]:
from google.colab import drive
import shutil
import os
import pandas as pd
import optuna
import subprocess

drive.mount('/content/drive')

drive_folder = '/content/drive/MyDrive/TES_results/'
os.makedirs(drive_folder, exist_ok=True)
csv_file = 'TES_fine_tuning_loglikelihood_results.csv'
drive_csv_path = os.path.join(drive_folder, csv_file)


if not os.path.exists(csv_file):
    columns = [
      'epoch', 'batch_size', 'd_model',
      'd_inner', 'd_k', 'd_v',
      'n_head', 'n_layers',
      'dropout', 'lr',
      'event_interest',
      'val_loglikelihood'
    ]
    df = pd.DataFrame(columns=columns)
    df.to_csv(csv_file, index=False)
    shutil.copy(csv_file, drive_csv_path)

def save_to_drive():
    shutil.copy(csv_file, drive_csv_path)
    print(f"Updated {csv_file} saved to Google Drive at {drive_csv_path}.")

def fine_tune_model(epoch, batch_size, d_model, d_inner, d_k, d_v, n_head,
                    n_layers, dropout, lr, event_interest):
    print(f"Running fine-tuning with parameters: "
          f"epoch={epoch}, batch_size={batch_size}, d_model={d_model}, "
          f"d_inner={d_inner}, d_k={d_k}, d_v={d_v}, n_head={n_head}, n_layers={n_layers}, "
          f"dropout={dropout}, lr={lr}, event_interest={event_interest}")

    def run_command(command):
        print(f"Running command: {' '.join(command)}")
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        output = result.stdout
        return output

    command = [
        'python', '/content/drive/MyDrive/causality-guided-Transformer/TES_Main_MAVEN_ERE.py',
        '-data', '/content/drive/MyDrive/causality-guided-Transformer/data/MAVEN_ERE/',
        '-epoch', str(epoch),
        '-batch_size', str(batch_size),
        '-d_model', str(d_model),
        '-d_inner', str(d_inner),
        '-d_k', str(d_k),
        '-d_v', str(d_v),
        '-n_head', str(n_head),
        '-n_layers', str(n_layers),
        '-dropout', str(dropout),
        '-lr', str(lr),
        '-event_interest', str(event_interest)
    ]

    output = run_command(command)

    train_loglikelihood = None
    val_loglikelihood = None
    test_loglikelihood = None

    for line in output.splitlines():
        if '(Training)' in line:
            train_loglikelihood = float(line.split('loglikelihood:')[-1].split(',')[0].strip())
        elif '(Validation)' in line:
            val_loglikelihood = float(line.split('loglikelihood:')[-1].split(',')[0].strip())
        elif '(Test)' in line:
            test_loglikelihood = float(line.split('loglikelihood:')[-1].split(',')[0].strip())
        elif 'F1 Score' in line:
            f1_score = float(line.split(':')[-1].strip())

    return val_loglikelihood, f1_score

def objective(trial):
    epoch = trial.suggest_categorical('epoch', [25, 35, 45])
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    d_model = trial.suggest_categorical('d_model', [128, 256, 512])
    d_inner = trial.suggest_categorical('d_inner', [256, 512, 1024])
    d_k = trial.suggest_categorical('d_k', [32, 64, 128])
    d_v = trial.suggest_categorical('d_v', [32, 64, 128])
    n_head = trial.suggest_categorical('n_head', [4, 8, 16])
    n_layers = trial.suggest_categorical('n_layers', [4, 6, 8])
    dropout = trial.suggest_float('dropout', 0.1, 0.3)
    lr = trial.suggest_loguniform('lr', 1e-5, 5e-4)
    event_interest = trial.suggest_categorical('event_interest', [7])

    val_loglikelihood = fine_tune_model(epoch, batch_size,
                                        d_model, d_inner,
                                        d_k, d_v, n_head,
                                        n_layers, dropout,
                                        lr, event_interest)

    trial_result = {
        'epoch': epoch,
        'batch_size': batch_size,
        'd_model': d_model,
        'd_inner': d_inner,
        'd_k': d_k,
        'd_v': d_v,
        'n_head': n_head,
        'n_layers': n_layers,
        'dropout': dropout,
        'lr': lr,
        'event_interest': event_interest,
        'val_loglikelihood': val_loglikelihood
    }

    trial_df = pd.DataFrame([trial_result])
    trial_df.to_csv(csv_file, mode='a', header=False, index=False)

    save_to_drive()

    return val_loglikelihood

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=300)

print("Best hyperparameters:", study.best_params)
print("Best validation loglikelihood:", study.best_value)
